
</h1> <h1 style='direction:rtl; font-family: "B Lotus";'> تمرین دوم - پردازش شعرهای با لهجه‌ی شیرازی

    
  

<h2 style='direction:rtl;font-family: "B Lotus";'> داده‌ی ورودی </h2> 

<p style='direction:rtl;font-family: "B Lotus";' >
</p>


In [ ]:
path_dir = "."


In [ ]:
! pip install pandas
! pip install hazm
! pip install transformers
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 8.3 MB/s 
     |████████████████████████████████| 1.4 MB 57.2 MB/s 
     |████████████████████████████████| 233 kB 66.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=ddbbe1b0064828728764ad7741952c30584435c9958055d9ed2582cd1cbb3e1f
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154526 sha256=e3a285cc6b37445a4a8d7d7a5a25d9f9736be16436ba8f8677f46e1c4cb94063
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 

In [ ]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import os
import re
import glob
from __future__ import unicode_literals
from hazm import *
from nltk import FreqDist
import itertools
import pandas as pd

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
class Cleaner:
    def __init__(self):
        self.normalizer = Normalizer()
        # self.stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('data/stopwords.txt','r','utf-8').readlines()]
        self.file_range = None


    def read_poems(self, file_patterns: str, file_range: tuple, normalize=False):  
        file_range = range(*file_range) if file_range is not None else None 
        file_names = glob.glob(file_patterns)
        mesra_collection = []
        for file_name in file_names:
            if normalize:
                mesra_collection += [self.normalizer.normalize(x) for x in codecs.open(file_name,'rU','utf-8').readlines()]
            else:
                mesra_collection += codecs.open(file_name,'rU','utf-8').readlines()[2:]
        return mesra_collection
    

    def read_documents(self):
        poems = self.read_poems(self.file_pattern, self.file_range)
        cleaned = self.clean_data(poems)
        return ' \n '.join(cleaned)
    

In [ ]:
path=f"{path_dir}/ferdousi.txt"
cleaner = Cleaner()
mesras = cleaner.read_poems(file_patterns=path, file_range=None, normalize=False)
print(len(mesras))

99217


In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

# v3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# model = TFAutoModel.from_pretrained(model_name_or_path)  For TF
model = AutoModel.from_pretrained(model_name_or_path)

text = "ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenizer.tokenize(text)


Downloading: 100%|██████████| 452M/452M [03:31<00:00, 2.23MB/s]   
Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-b

['ما',
 'در',
 'هوش',
 '[ZWNJ]',
 'واره',
 'معتقدیم',
 'با',
 'انتقال',
 'صحیح',
 'دانش',
 'و',
 'آ',
 '##گاهی',
 '،',
 'همه',
 'افراد',
 'میتوانند',
 'از',
 'ابزارهای',
 'هوشمند',
 'استفاده',
 'کنند',
 '.',
 'شعار',
 'ما',
 'هوش',
 'مصنوعی',
 'برای',
 'همه',
 'است',
 '.']

In [ ]:
from transformers import (
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelWithLMHead,
    BertModel
)


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer, file_path=train_path, block_size=256
    )

    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    return train_dataset, test_dataset, data_collator


# Freezing the lower layers increases the training speed and reduces the memory requirement.
# Depending on your task you may want to freeze all layers and train addition layers that you are adding to the model
# or unfreeze as many layers that you can affort training with a reasonable batchsize.
def freeze_lower_layers():
    for param in model.base_model.parameters():
        param.requires_grad = False

    for param in (
        model.base_model.h[23].parameters() or model.base_model.h[22].parameters()
    ):
        param.requires_grad = True


# load model
model = AutoModelWithLMHead.from_pretrained("bolbolzaban/gpt2-persian")

# freeze lower layers and only train top layers
freeze_lower_layers()

# load dataset
tokenizer = AutoTokenizer.from_pretrained("bolbolzaban/gpt2-persian")
train_dataset, test_dataset, data_collator = load_dataset(
    "ferdousi.txt", "test.txt", tokenizer
)
print(type(train_dataset))


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


<class 'transformers.data.datasets.language_modeling.TextDataset'>


In [ ]:
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')
model = GPT2LMHeadModel.from_pretrained('bolbolzaban/gpt2-persian')
freeze_lower_layers()
generator = pipeline('text-generation', model, tokenizer=tokenizer, config={'max_length':256})
sample = generator('در یک اتفاق شگفت انگیز، پژوهشگران')

loading configuration file https://huggingface.co/bolbolzaban/gpt2-persian/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/55af8b32edc9c441878906a22de9b0c79dce98654714626407f7ea3ee7fc7349.1f2d676354b04ed818190819fee0d7d2deb97cfe456b804edfea24e6a8053fd5
Model config GPT2Config {
  "_name_or_path": "bolbolzaban/gpt2-persian",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_dropout": 0.1,
  "attn_pdrop": 0.1,
  "beta1": 0.9,
  "beta2": 0.98,
  "bos_token_id": 8,
  "data_path": "",
  "embd_pdrop": 0.1,
  "embed_dropout": 0.1,
  "eos_token_id": 9,
  "epsilon": 1e-09,
  "eval_batch_size": 128,
  "eval_steps": 10,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "iterations": 1000,
  "layer_norm_epsilon": 1e-05,
  "line_count": 1,
  "lr": 0.00025,
  "max_steps": 3600000,
  "model": "GPT2",
  "model_type": "gpt2",
  "n_ctx": 256,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_

In [ ]:
generator(' توانا بود هرکه')

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


[{'generated_text': ' توانا بود هرکه او را یاری کرد.'}]

In [ ]:
print(sample)

[{'generated_text': 'در یک اتفاق شگفت انگیز، پژوهشگران آمریکایی ، موفق به کشف آن شدند.'}]


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts = train_test_split(train_dataset)

In [ ]:
print(type(train_texts))

<class 'list'>


In [ ]:
print(type(val_texts))
print(type(train_dataset))

<class 'list'>
<class 'transformers.data.datasets.language_modeling.TextDataset'>


In [ ]:
print(len(train_texts))
print(len(val_texts))
print(len(train_dataset))

1895
632
2527


In [ ]:
# train
training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    # Set the batch size to a maximum value that could fit into GPU memory,
    # for example 12 is the largest batch size that could work on a 6gb GPU when training the last to layers
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    eval_steps=1000,
    save_steps=1000,
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_texts,
    eval_dataset=val_texts,
)


trainer.train()

# save
trainer.save_model()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


RuntimeError: ignored

<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین سوم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "تکمیل مصراع دوم با رعایت وزن شعر" را انتخاب کردیم. در این تمرین قصد داریم به کمک مدل‌های زبانی مناسب، چند کلمه به عنوان یک مصرع از بیت را به عنوان ورودی بگیریم و چند کلمه به عنوان مصرع دوم این بیت را خروجی دهیم. در این تمرین ما از وبسایت گنجور برای جمع‌آوری داده کمک گرفتیم و از چندین قالب شعری استفاده کردیم، زیرا قافیه داشتن یا نداشتن به عنوان ورودی داده می‌شود. اگر نیاز به هم‌قافیه بودن دو مصراع بود، از قالب مثنوی نظیر شاهنماه‌ی فردوسی کمک می‌گیریم؛ و اگر نیاز به هم‌قافیه بودن دو مصراع نبود می‌توانیم از قالب قطعه استفاده کنیم. <br>
    برای ساخت چنین سیستمی از چند مدل زبانی استفاده می‌کنیم: مدل n-gram، مدل encoder-decoder با یک شبکه LSTM به عنوان encoder، و مدلی پیرو مکانیسم توجه.
</div>

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 1. دوباره‌نویسی داده </h3>
    از آنجایی که در داده‌ی گرفته شده، ابتدا و انتهای ابیات و مصراع‌ها مشخص نبود، به کمک کد زیر این داده را بازنویسی کردیم. ابتدا و انتهای ابیات با __BOB__ و __EOB__ و ابتدا و انتهای مصراع‌ها با __BOM__ و __EOM__ مشخص گردید.
</div>

In [1]:
path = 'ferdousi_norm.txt'

beyt_file = ''
with open(path, 'r', encoding="utf-8") as fp:
    lines = fp.readlines()
    for i in range(0, len(lines) - 1, 2):
        mesra1 = '__BOM__ ' + lines[i].strip() + ' __EOM__ '
        mesra2 = '__BOM__ ' + lines[i + 1].strip() + ' __EOM__'
        b = '__BOB__ ' + mesra1.strip() + ' ' + mesra2.strip() + ' __EOB__\n'
        beyt_file += b
        
with open('f_beyt.txt', 'w', encoding="utf-8") as fp:
    fp.write(beyt_file)